# Chapter 5: Theorem Proving

> Testing can be used to show the presence of bugs, but never to show their absence. - Edsger W. Dijkstra

Unit tests (Ch.3) and property-based tests (Ch.4) give us confidence, but they still sample the input space. To climb the next rung on the ladder of rigor, we turn to **theorem proving**, a formal verification technique.

In this chapter we will:

 - Use a theorem prover to check whether properties hold for all possible inputs (within specified ranges).
 - Encode the *what* of our computations, rather than the *how*, to make reasoning possible.
 - Explore both the strengths and the limits of theorem proving, and see how it complements testing.

Theorem proving provides guarantees that testing alone cannot match. But these guarantees come at a cost:

 - **Encoding effort:** you must write down precise specifications and constraints.
 - **Computational expense:** solvers can become slow or intractable as problem size grows.

In practice, theorem proving shines on select **critical** routines (where correctness is paramount) and core algorithms (where tests may miss subtle edge cases).

## Small Scope Hypothesis

Formal methods may not scale to arbitrarily large programs or input sizes. For example, we may only be able to prove properties of our heat equation solver for small meshes (e.g., 10-100 points). But this is still useful in practice, thanks to the **small scope hypothesis** (coined by Daniel Jackson):

> A high proportion of bugs can be found by checking a program for all inputs within some small scope.

Even such small proofs can expose bugs or confirm that a property is mathematically guaranteed under the right conditions.

## The Z3 Theorem Prover

Z3 is a popular theorem prover developed by Microsoft Research. Workflow:
 - Declare symbolic variables (like `x`, `y`, `res`).
 - State constraints: preconditions `P` and the effects of the `code`.
 - Assert the postconditions `Q` you want to hold.
 - Ask Z3 to search for a counterexample to `P and code implies Q`.
   - If it finds a counterexample, the property is not universally valid.
   - If no counterexample exists, the property is proven (for the chosen theory and input size).

Unlike testing, which explores a handful of examples, Z3 can reason over all inputs in one shot.

We’ll use Z3’s Python API, z3py.

### Declaring Variables

Z3 supports many symbolic types: integers, reals, booleans, bit-vectors, arrays, strings, and more. These are not ordinary variables but symbols that range over all possible values of their type, constrained only by what you add later.

In [1]:
from z3 import Bool, Int, Real

p = Bool('p')
i = Int('i')
x = Real('x')

Multiple variables can be declared at once:

In [2]:
from z3 import Bools, Ints, Reals

p, q = Bools('p q')
i, j, k = Ints('i j k')
x, y, z, res = Reals('x y z res')

You can also create arrays programmatically:

In [3]:
F = [Real(f'F{i}') for i in range(10)]

Here:
 - The strings on the RHS are names Z3 uses internally to identify the variables.
 - The Python variables on the LHS are what you’ll use in your constraints and properties.

They don’t have to match, but keeping them consistent avoids confusion.

In [4]:
x, y, res

(x, y, res)

In [5]:
F[0], F[1], F[2]

(F0, F1, F2)

### Declarative Style

Z3 is declarative: you describe what must hold, not how to compute.
That means:
 - You don't *assign* values to variables.
 - Instead, you *constrain* variables with equations and (in)equalities.

For example:

In [6]:
x == 0        # a Boolean formula (not an assignment)
y >= x + 1    # another Boolean formula

y >= x + 1

On their own, these Boolean formulas don’t do anything, but once added to a solver, Z3 will try to find values of `x` and `y` that make them true.

### The Z3 Solver

A solver collects constraints and checks *satisfiability*.

In [7]:
from z3 import Solver

# Instantiate a solver object
s = Solver()

# Add constraints to the solver
s.add(x == 0)
s.add(y == x + 1)

If the constraints are contradictory, the solver will report `unsat` (unsatisfiable). If they can be satisfied, it will report `sat` (satisfiable).

In [8]:
s.check()

sat

#### sat vs. unsat

In [10]:
from z3 import sat, unsat

 - **Satisfiable** means there is at least one assignment of values to the variables that satisfies all the constraints.
 - **Unsatisfiable** means there is no assignment of values to the variables that satisfies all the constraints.

### Logical Connectives

Z3 supports standard logical connectives:

In [11]:
from z3 import And, Or, Not, Implies, If


 - `And(a, b, ...)` - all must be true
 - `Or(a, b, ...)` - at least one must be true
 - `Not(a)` - negation
 - `Implies(a, b)` - if `a` is true, then `b` must be true
 - `If(a, b, c)` - if `a` is true, then `b`, else `c`

### From Specs to Proofs

Let's revisit our simple `div` function:

In [12]:
def div(x, y):
    assert y != 0           # precondition
    res = x / y             # code
    assert res * y == x     # postcondition
    return res

We can’t feed this Python directly to Z3, but we can encode the same idea declaratively:

In [13]:
def div(res, x, y):
    return And(
        y != 0,          # precondition
        res == x / y,    # code
        res * y == x     # postcondition
    )

Now, we can check:

In [14]:
# Create a solver:
s = Solver()

# *call* div(res, x, y) with symbolic variables:
s.add(div(*Reals('res x y')))

# Check if the div constraints are satisfiable:
s.check()

sat

This just tells us “there exists at least one input where it works.”

To prove it works for all inputs, we flip the logic:

In [15]:
from z3 import Not

def div(res, x, y):
    return And(
        y != 0,
        res == x / y,
        Not(res * y == x)   # negated postcondition
    )

In [16]:
# Create a solver:
s = Solver()

# *call* div(res, x, y) with symbolic variables:
s.add(div(*Reals('res x y')))

# Check if the div constraints are satisfiable:
s.check()

unsat

Now Z3 confirms: there are no inputs (with y != 0) that break the postcondition, so the property holds universally, at least in real arithmetic.

### Real vs Floating-Point

Note that we used Reals. Real arithmetic is exact; floating-point is not.
That's why Z3 didn't *see* the earlier 7/25 counterexample.
 - Reals are good for proving mathematical properties of algorithms.
 - Floating-point (`FP` in Z3) lets you model IEEE-754 semantics, but is heavier and harder to reason about exhaustively.

For our purposes, reasoning in reals is an advantage: it abstracts away round-off details and lets us focus on the algorithm’s conceptual correctness. Floating-point quirks can be handled later with testing or targeted FP analysis.

### Why this matters

Even with these limitations, Z3 gives you something testing cannot:
 - It can prove absence of counterexamples within a theory.
 - It can quickly expose missing or too-weak preconditions.
 - It can find subtle edge cases that tests might miss.

In short, testing may show the presence of bugs; theorem proving will show their absence within specified bounds.


## Back to the Heat Equation Solver

We’ll now reason about our finite-volume 1-D diffusion solver. 

So far we've worked *imperatively*, writing code that *does* things step-by-step. To use Z3, we need to switch to a more *declarative* style, describing *what* the code should achieve rather than *how* it does so.

### Telescoping Property

As a first step, let's prove the **telescoping property of divergence**: The sum of the divergence over all cells equals the net flux through the boundaries.

$\qquad
\sum_{i=0}^{N-1} (\nabla \cdot F)_i = F_0 - F_N
\qquad$


To do so, we'll re-express the divergence procedure as a constraint generator: a function that takes Z3 variables and returns Z3 constraints representing the original procedure's behavior.

Recall the divergence function and the telescoping property (assertions omitted for brevity):

In [17]:
def divergence(c, f, dx):
    """Compute the divergence of face quantities (f) and store in (c)."""
    for i in range(len(c)):
        c[i] = (f[i] - f[i+1]) / dx

def telescoping(c, f, dx):
    """Check the finite volume telescoping property."""
    total_divergence = sum(c) * dx
    boundary_flux = f[0] - f[-1]
    return total_divergence == approx(boundary_flux)

The Z3 versions of these functions:

In [18]:
def divergence(c, f, dx):
    """Compute the divergence of face quantities (f) and store in (c)."""
    N = len(c)
    return [c[i] == (f[i] - f[i+1]) / dx for i in range(N)]

def telescoping(c, f, dx):
    """Check the finite volume telescoping property."""
    N = len(c)
    total_divergence = sum(c[i] for i in range(N)) * dx
    boundary_flux = f[0] - f[-1]
    return total_divergence == boundary_flux

Notice we no longer need the `approx` helper since Z3 handles real arithmetic exactly.

To call these functions, we need to pass Z3 variables:

In [19]:
N = 3 # number of cells
dx, dt, kappa = Reals('dx dt kappa')
f = [Real(f'f{i}') for i in range(N+1)]
c = [Real(f'c{i}') for i in range(N)]

Now we can set up a solver to check the telescoping property:

In [20]:
s = Solver()
s.add(dx > 0, dt > 0, kappa > 0)  # Preconditions:physical constraints
s.add(divergence(c, f, dx))       # code: divergence
s.add(Not(telescoping(c, f, dx))) # Postcondition: telescoping property
s.check()

unsat

The Z3 solver confirms `unsat`: there are no inputs that violate the telescoping property, so it holds universally for all valid inputs where N=3. Let's attempt to increase N and see how far we can go before Z3 struggles.

In [21]:
def prove_telescoping(N):
    dx = Real('dx')
    f = [Real(f'f{i}') for i in range(N+1)]
    c = [Real(f'c{i}') for i in range(N)]

    s = Solver()
    s.add(dx > 0)                    # Preconditions: physical constraints
    s.add(divergence(c, f, dx))      # code: divergence
    s.add(Not(telescoping(c, f, dx)))# Postcondition: telescoping property
    return s.check() == unsat

In [22]:
import time

i = 3
while True:
    start_time = time.time()
    result = prove_telescoping(i)
    elapsed_time = time.time() - start_time
    print(f'i={i}, result={result}, time={elapsed_time}')
    if elapsed_time > 2:  # Stop if it takes too long
        break
    i *= 2

i=3, result=True, time=0.007091045379638672
i=6, result=True, time=0.008424043655395508
i=12, result=True, time=0.011858940124511719
i=24, result=True, time=0.02184295654296875
i=48, result=True, time=0.05122995376586914
i=96, result=True, time=0.1603546142578125
i=192, result=True, time=0.7051713466644287
i=384, result=True, time=3.7941009998321533


### Core Procedures as Constraints

Let's re-express all of the core procedures of the heat equation solver 
as **constraint generators**: functions that take Z3 variables and return Z3 constraints representing
the original procedures' behavior.

In [23]:
def apply_bc(f, bc):
    """Apply BCs by overriding first and last face quantities (f)."""
    return [f[0] == bc[0], f[-1] == bc[1]]

def diffusive_flux(f, c, kappa, dx):
    """Given a cell field (c), compute the diffusive flux (f)."""
    N = len(c)
    return [f[i] == -kappa * (c[i] - c[i-1]) / dx for i in range(1, N)]

def divergence(c, f, dx):
    """Compute the divergence of face quantities (f) and store in (c)."""
    N = len(c)
    return [c[i] == (f[i] - f[i+1]) / dx for i in range(N)]

def step_heat_eqn(u0, u1, divF, dt):
    """Advance cell field u by one time step using explicit Euler method."""
    N = len(u0)
    return [u1[i] == u0[i] + dt * divF[i] for i in range(N)]

We'll use these declarative versions of the core computations to model the heat equation time step and prove properties about it.

Having specified these core computations declaratively, now let's define the symbolic variables. In unit tests, we handpicked specific inputs. In property-based tests, we sampled many inputs randomly. Here, we define symbolic variables that represent *all possible* inputs within specified bounds.

In [28]:
# Define the parameters
dx, dt, kappa = Reals("dx dt kappa")

# Boundary conditions
bc = Reals("bc0 bc1")

# Define the vectors based on mesh size N
def allocate_vectors(N):
    u_old = [Real(f"u_old_{i}") for i in range(N)]
    u_new = [Real(f"u_new_{i}") for i in range(N)]
    divF = [Real(f"divF_{i}") for i in range(N)]
    F = [Real(f"F{i}") for i in range(N+1)]
    return u_old, u_new, divF, F

### Proving a property

Below, we define a helper function to run the Solver `check` method. If the solver returns `unsat`, the property holds; otherwise, we print a counterexample.

In [71]:
def prove(s):
    if s.check() == unsat:
        print("Property holds.")
    else:
        print("Property does NOT hold!")
        print("Below is a counterexample:")
        print(s.model())

We are now ready to state and prove more properties about our heat equation solver.

### Conservation

Recall the below encoding of the **conservation property** from the previous chapters:

In [60]:
def heat_is_conserved(u_old, u_new, dt, dx, bc):
    """Check if heat is conserved."""
    lhs = sum(u_new) * dx
    rhs = sum(u_old) * dx + dt * (bc[0] - bc[1])
    return lhs == rhs

In [64]:
def prove_conservation(N=5):
    s = Solver()

    # Allocate vectors
    u_old, u_new, divF, F = allocate_vectors(N)

    # Positive, arbitrary parameters
    s.add(dx > 0, dt > 0, kappa > 0)

    # Model the heat equation time step
    s.add(apply_bc(F, bc))
    s.add(diffusive_flux(F, u_old, kappa, dx))
    s.add(divergence(divF, F, dx))
    s.add(step_heat_eqn(u_old, u_new, divF, dt))

    # Attempt to disprove conservation
    s.add(Not(heat_is_conserved(u_old, u_new, dt, dx, bc)))

    prove(s)

In [65]:
prove_conservation()

Property holds.


The `unsat` result indicates that Z3 is unable to find any counterexample that violates the conservation property. As such, we have proven that the conservation property holds for all possible inputs (for real arithmetic and N=5).

### Symmetry

If the initial state is symmetric about the center and the boundary fluxes are equal and opposite (antisymmetric),
then the updated state is also symmetric:
 
 - Mesh length $N$ is chosen odd so there's a unique center.
 - We encode symmetry as $u_{i} = u_{N-1-i}$ for all $i$.
 - Antisymmetric BCs: $F_0 = -F_{N}$.

In [66]:
def is_symmetric(u):
    N = len(u)
    return And([u[i] == u[N-1-i] for i in range(N)])

def prove_symmetry_preserved(N=5):
    s = Solver()

    # Allocate vectors
    u_old, u_new, divF, F = allocate_vectors(N)

    # Positive, arbitrary parameters
    s.add(dx > 0, dt > 0, kappa > 0)

    # symmetric initial condition
    s.add(is_symmetric(u_old))

    # Apply antisymmetric, arbitrary BCs
    s.add(bc[0] == -bc[1])

    # Model the heat equation time step
    s.add(apply_bc(F, bc))
    s.add(diffusive_flux(F, u_old, kappa, dx))
    s.add(divergence(divF, F, dx))
    s.add(step_heat_eqn(u_old, u_new, divF, dt))

    # Attempt to disprove symmetry preservation
    s.add(Not(is_symmetric(u_new)))

    prove(s)

In [67]:
prove_symmetry_preserved()

Property holds.


### Monotonicity

Diffusion smooths. One way to express this intuition discretely is through monotonicity preservation:
if a temperature profile is nondecreasing at the start of a step, it should remain so after the update.
In other words, diffusion should not create new inversions.

In [68]:
def is_nondecreasing(u):
    return And([u[i] <= u[i+1] for i in range(len(u)-1)])

def prove_monotonicity(N=3):
    s = Solver()

    # Allocate vectors
    u_old, u_new, divF, F = allocate_vectors(N)

    # Positive, arbitrary parameters
    s.add(dx > 0, dt > 0, kappa > 0)
    #s.add(dt <= dx*dx / (2*kappa))  # r <= 1/2

    # insulated boundaries
    s.add(bc[0] == 0, bc[1] == 0)

    # Model the heat equation time step
    s.add(apply_bc(F, bc))
    s.add(diffusive_flux(F, u_old, kappa, dx))
    s.add(divergence(divF, F, dx))
    s.add(step_heat_eqn(u_old, u_new, divF, dt))

    # nondecreasing initial profile
    s.add(is_nondecreasing(u_old))

    # ask Z3 to find a counterexample that breaks monotonicity
    s.add(Not(is_nondecreasing(u_new)))

    prove(s)

In [69]:
prove_monotonicity()

Property does not hold.
Below is a counterexample:
[u_new_1 = 0,
 kappa = 2,
 u_new_2 = -1/2,
 u_old_1 = 0,
 F3 = 0,
 F0 = 0,
 dt = 1,
 dx = 1,
 bc1 = 0,
 u_old_2 = 1/2,
 u_new_0 = 1/2,
 divF_1 = 0,
 F2 = -1,
 bc0 = 0,
 u_old_0 = -1/2,
 divF_2 = -1,
 divF_0 = 1,
 F1 = -1,
 /0 = [(-1, 1) -> -1, (1, 1) -> 1, else -> 0]]


Above output reveals that the monotonicity property does NOT hold universally. The counterexample provided by Z3 shows specific values for the parameters and initial conditions where the property fails. This insight can guide us in refining our model or understanding the limitations of the diffusion process under certain conditions:

Looking closely at the counterexample, we see that `dt` is quite large relative to `dx` and `kappa`, leading to instability in the diffusion process. Let's introduce the stability condition and see if that helps prove monotonicity.

In [72]:
def is_nondecreasing(u):
    return And([u[i] <= u[i+1] for i in range(len(u)-1)])

def prove_monotonicity(N=3):
    s = Solver()

    # Allocate vectors
    u_old, u_new, divF, F = allocate_vectors(N)

    # Positive, arbitrary parameters
    s.add(dx > 0, dt > 0, kappa > 0)
    s.add(dt <= dx*dx / (2*kappa))  # r <= 1/2

    # insulated boundaries
    s.add(bc[0] == 0, bc[1] == 0)

    # Model the heat equation time step
    s.add(apply_bc(F, bc))
    s.add(diffusive_flux(F, u_old, kappa, dx))
    s.add(divergence(divF, F, dx))
    s.add(step_heat_eqn(u_old, u_new, divF, dt))

    # nondecreasing initial profile
    s.add(is_nondecreasing(u_old))

    # ask Z3 to find a counterexample that breaks monotonicity
    s.add(Not(is_nondecreasing(u_new)))

    prove(s)

In [73]:
prove_monotonicity()

Property holds.


With the added stability condition, Z3 confirms that the monotonicity property holds universally for the heat equation time step under the specified conditions. This demonstrates how Z3 can help us identify necessary conditions for properties to hold.

### Positivity

We expect that, given a non-negative initial temperature distribution and non-negative boundary conditions, the temperature remains non-negative after one time step. Below is an expression of this property in Z3:

In [76]:
def all_positive(c):
    """All cell values are non-negative."""
    N = len(c)
    return And([c[i] >= 0 for i in range(N)])

def prove_positivity(N=4):
    s = Solver()

    # Allocate vectors
    u_old, u_new, divF, F = allocate_vectors(N)

    # Positive, arbitrary parameters
    s.add(dx > 0, dt > 0, kappa > 0)
    s.add(dt <= dx*dx / (2*kappa))  # r <= 1/2
    s.add(bc[0] >= 0, bc[1] <= 0)

    # Model the heat equation time step
    s.add(apply_bc(F, bc))
    s.add(diffusive_flux(F, u_old, kappa, dx))
    s.add(divergence(divF, F, dx))
    s.add(step_heat_eqn(u_old, u_new, divF, dt))

    # non-negative initial profile and BCs
    s.add(all_positive(u_old))

    # ask Z3 to find a counterexample that produces negative temperature
    s.add(Not(all_positive(u_new)))

    prove(s)

In [77]:
prove_positivity()

Property holds.



## What we just did

We re-expressed the solver’s core procedures as constraint generators and used Z3 to:

 - Prove universal identities (telescoping, conservation).
 - Find counterexamples when preconditions are missing (positivity).
 - Confirm fixes by adding the right guards (stability)

This completes the ladder:

 - Ch.1: working baseline (code & fix)
 - Ch.2: abstractions & specs
 - Ch.3: unit tests
 - Ch.4: property-based tests
 - Ch.5: theorem proving

In practice, you’d mix these: run unit/property tests in CI, and apply theorem proving judiciously on small kernels where stronger guarantees pay off. Together, they move scientific software toward explicit reasoning and trustworthy results, without heavy ceremony.

---

R3Sw tutorial by Alper Altuntas (NSF NCAR). Guest lecture by **Philip Zucker** (Draper). Sponsored by the BSSw Fellowship Program. © 2025.

Cite as: Alper Altuntas, Philip Zucker, Deepak Cherian, Adrianna Foster, Manish Venumuddula, and Helen Kershaw. (2025). *"Rigor and Reasoning in Research Software (R3Sw) Tutorial."* https://www.alperaltuntas.com/R3Sw